In [2]:
import google.generativeai as genai
import sys

# Configure Gemini with your API key
genai.configure(api_key="")  # Replace with your key

def get_completion_from_messages_gemini(messages, 
                                        model_name="models/gemini-1.5-flash", 
                                        temperature=0, max_tokens=500):
    model = genai.GenerativeModel(model_name)
    chat = model.start_chat()

    # Convert structured messages into a single prompt
    prompt = ""
    for msg in messages:
        if msg["role"] == "system":
            prompt += "[SYSTEM]\n" + msg["content"] + "\n\n"
        elif msg["role"] == "user":
            prompt += "[USER]\n" + msg["content"] + "\n\n"
        elif msg["role"] == "assistant":
            prompt += "[ASSISTANT]\n" + msg["content"] + "\n\n"

    response = chat.send_message(prompt, generation_config={
        "temperature": temperature,
        "max_output_tokens": max_tokens
    })

    return response.text.strip()

# Core logic
delimiter = "####"
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product cateogry doesn't count. 

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.
All available products: 
1. Product: TechPro Ultrabook
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: A sleek and lightweight ultrabook for everyday use.
   Price: $799.99

2. Product: BlueWave Gaming Laptop
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: A high-performance gaming laptop for an immersive experience.
   Price: $1199.99

3. Product: PowerLite Convertible
   Category: Computers and Laptops
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop with a responsive touchscreen.
   Price: $699.99

4. Product: TechPro Desktop
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: A powerful desktop computer for work and play.
   Price: $999.99

5. Product: BlueWave Chromebook
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook for everyday tasks.
   Price: $249.99

Step 3:{delimiter} If the message contains products \
in the list above, list any assumptions that the \
user is making in their \
message e.g. that Laptop X is bigger than \
Laptop Y, or that Laptop Z has a 2 year warranty.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on your \
product information. 

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of \
5 available products, as these are the only 5 \
products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

# First query
user_message = f"""
by how much is the BlueWave Chromebook more expensive \
than the TechPro Desktop"""

messages =  [  
    {'role':'system', 'content': system_message},    
    {'role':'user', 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages_gemini(messages)
print(response)

# Second query
user_message = f"""do you sell tvs"""
messages =  [  
    {'role':'system', 'content': system_message},    
    {'role':'user', 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages_gemini(messages)

print(response)

# Extract final customer-friendly response from last part
try:
    final_response = response.split(delimiter)[-1].strip()
except Exception:
    final_response = "Sorry, I'm having trouble right now, please try asking another question."

print(final_response)


Step 1:#### The user is asking about the price difference between two specific products: the BlueWave Chromebook and the TechPro Desktop.

Step 2:#### Both the BlueWave Chromebook and the TechPro Desktop are in the list of available products.

Step 3:#### The user is not making any assumptions about the features or specifications of the products, only comparing their prices.

Step 4:#### No assumptions to verify.

Response to user:#### Hi there!  The TechPro Desktop is priced at $999.99, and the BlueWave Chromebook costs $249.99.  Therefore, the TechPro Desktop is $750 more expensive than the BlueWave Chromebook.
Step 1:#### The user is not asking about specific products from the list.

Step 2:#### The user's query is not about any of the listed products.

Step 3:#### Not applicable.

Step 4:#### Not applicable.

Response to user:#### I'm sorry, but we don't currently sell TVs.  We specialize in computers and laptops.  Is there anything else I can help you with?
I'm sorry, but we don't